Jupyter Notebook For ReadMyMind, A CS 125 @ Illinois MP7 Project by Isaac Park and Mihir Pandya.

In [38]:
import tweepy
import pandas as pd
from textblob import TextBlob
from datetime import datetime
import pprint
import numpy as np

username = 'realDonaldTrump' # working example: Donald Trump

auth = tweepy.OAuthHandler('1X5fCqPl7yVvYxQjQJwkvavFD', 'NXbTDPP3HxlXOL5dWdCegEP09odLAkxUWlyRvZqXxtAtdX597G')
auth.set_access_token('925495606931546112-mn3Hda41LsZhbYAKJtddL7TulRKucuj', 'lvCFqSLv5YvOGzCINH6JZ5cBI1CEkPKrRioBn5Iuec3Tt')
api = tweepy.API(auth)
    
tweets_df = pd.DataFrame({
    'Timestamp': (),
    'Likes': (),
    'Retweets': (),
    'Text': (),
    'Sentences': (),
    'Sentiment_Total': (),
    'Keywords': ()
})

tweets_df = tweets_df[['Timestamp', 'Likes', 'Retweets', 'Text', 'Sentences', 'Sentiment_Total', 'Keywords']]
    
recent_tweets = api.user_timeline(screen_name = username, count=100, tweet_mode="extended") # analyzing 100 tweets
for status in recent_tweets:
    test = status.full_text
    if test[:2] != 'RT': # removing retweets made by the user
        status_data = pd.Series([status.created_at, status.favorite_count, status.retweet_count, status.full_text], 
                                index=['Timestamp', 'Likes', 'Retweets', 'Text'])
    tweets_df = tweets_df.append(status_data, ignore_index = True)
    
tweets_df = tweets_df.drop_duplicates(subset='Text') # just in case, remove any duplicate tweets
tweets_df = tweets_df.astype('object')

keywords_dict = {}

for i in range(len(tweets_df)):
    content = tweets_df.iloc[i]['Text']
    if 'http' in content:
        j = content.index('http')
        content = content[:j] # cleaning text of the tweet by removing the link at the end and newline characters
    content = content.replace('\n', '')
    tweets_df.iloc[i]['Text'] = content
    
    blob = TextBlob(content)
    tweets_df.iloc[i]['Sentiment_Total'] = blob.sentiment.subjectivity
    sentiments = {}
    
    for sent in blob.sentences: # generating sentiment polarity values for each sentence in the tweet
        sentiments[str(sent)] = sent.sentiment.subjectivity
        
    tweets_df.iloc[i]['Sentences'] = sentiments # insert dictionary of sentence: sentiment value into dataframe
    
    tweets_df.iloc[i]['Timestamp'] = tweets_df.iloc[i]['Timestamp'].to_pydatetime() # convert pandas.tslib.Timestamp object to datetime
    
    # Keyword extraction goes here
    filtered_words = blob.noun_phrases
#     print(filtered_words)
    temp = []
    
    for element in filtered_words:
        for x in range(len(filtered_words)):
#             print(filtered_words[x])
#             print(element)
            if element != filtered_words[x] and element in filtered_words[x]:
                temp.append(element)
                #filtered_words = [x for x in filtered_words if x != element]
    parts_of_speech = blob.tags
    for element in temp:
        filtered_words = [x for x in filtered_words if x != element]
    
    for x in range(len(parts_of_speech)):
        if (parts_of_speech[x])[1] == 'NN':
            enter = True
            for element in filtered_words:
                if (parts_of_speech[x])[0] in element:
                    enter = False
            if enter:
                if x > 0 and (parts_of_speech[x - 1])[1] == 'PRP$':
                    filtered_words.append((parts_of_speech[x])[0])
    parenthesis = []
    paren_init = 0
    loc_begin = blob.find("(", paren_init)
    loc_end = blob.find(")", paren_init)
    
    while loc_end >= 0:
        parenthesis.append(blob[loc_begin:loc_end])
        paren_init = loc_end + 1
        loc_begin = blob.find("(", paren_init)
        loc_end = blob.find(")", paren_init)
    #print(parenthesis)
    
    for element in filtered_words:
        for pelement in parenthesis:
            if element in pelement.lower():
                filtered_words = [x for x in filtered_words if x != element]
#     tweets_df.iloc[i]['Keywords'] = filtered_words

    for word in filtered_words:
        separated = TextBlob(word).words
        for j in separated:
            j = j.strip()
            if j.isalpha() and len(j) > 2:
                if j in keywords_dict:
                    keywords_dict[j][0] += 1
                    keywords_dict[j][1] += tweets_df.iloc[i]['Likes']
                    keywords_dict[j][2] += tweets_df.iloc[i]['Retweets']
                    keywords_dict[j][3] += tweets_df.iloc[i]['Sentiment_Total']
#                     sentiment_sum = 0
#                     for sent in tweets_df.iloc[i]['Sentences']:
#                         if j in sent:
#                             sentiment_sum += tweets_df.iloc[i]['Sentences'][sent]
#                     keywords_dict[j][3] += sentiment_sum
                else:
                    keywords_dict[j] = [1, tweets_df.iloc[i]['Likes'], tweets_df.iloc[i]['Retweets'], tweets_df.iloc[i]['Sentiment_Total']]
#                     sentiment_sum = 0
#                     for sent in tweets_df.iloc[i]['Sentences']:
#                         if j in sent:
#                             sentiment_sum += tweets_df.iloc[i]['Sentences'][sent]
#                     keywords_dict[j][3] = sentiment_sum

for key in keywords_dict:
    keywords_dict[key][1] = int(keywords_dict[key][1] / keywords_dict[key][0])
    keywords_dict[key][2] = int(keywords_dict[key][2] / keywords_dict[key][0])
    keywords_dict[key][3] = keywords_dict[key][3] / keywords_dict[key][0]

keywords_df = pd.DataFrame.from_dict(keywords_dict, orient='index')
keywords_df.columns = ['Frequency', 'Avg. Likes', 'Avg. Retweets', 'Avg. Sentiment']
keywords_df.index.name = 'Keywords'
keywords_df.reset_index(inplace = True)
keywords_df = keywords_df.sort_values(['Frequency'], ascending = [False], na_position = 'last')
keywords_df = keywords_df[:30]

keywords_df

,Keywords,Frequency,Avg. Likes,Avg. Retweets,Avg. Sentiment
25,great,17,74423,15508,0.632480
4,america,15,77764,19465,0.631233
2,fake,12,88798,23246,0.609117
35,pensacola,9,61735,13868,0.372297
209,cuts,9,72724,16913,0.517917
208,tax,9,61127,15095,0.470904
65,cnn,7,112497,29027,0.537169
17,news,7,97927,25942,0.581112
11,market,7,96695,21615,0.517571
6,bill,6,82313,18484,0.701984


Proposed method for keyword extraction:

1. Tokenize each word with part of speech tag. keep only proper nouns, nouns, adjectives, and verbs.
2. Score the nouns and proper nouns based on amount of surrounding adjectives and verbs (using more description tends to indicate importance).
3. Record frequency of each word; only keep words that occur above a certain number of times (frequency threshold). These will be our "keywords".
4. Put the list of keywords for each tweet into the 'Keywords' column of the dataframe.

Ideas for graphing the keywords/frequency/likes/retweets relationships:

1. y-axis: frequency, x-axis: keyword; simple bar graph of the top keywords

2. y-axis: likes/retweet count, x-axis: frequencies of keywords; scatter plot with each dot representing a keyword.

3. Simple pie chart to analyze the main content areas that said Twitter account comments on.




Written below is the function/method oriented version of the program. Different parts of the program are organized into multiple functions. -Isaac

In [3]:
import tweepy
import pandas as pd
from textblob import TextBlob
from datetime import datetime
import numpy as np

username = 'realDonaldTrump' # working example: Donald Trump
auth = tweepy.OAuthHandler('1X5fCqPl7yVvYxQjQJwkvavFD', 'NXbTDPP3HxlXOL5dWdCegEP09odLAkxUWlyRvZqXxtAtdX597G')
auth.set_access_token('925495606931546112-mn3Hda41LsZhbYAKJtddL7TulRKucuj', 'lvCFqSLv5YvOGzCINH6JZ5cBI1CEkPKrRioBn5Iuec3Tt')
api = tweepy.API(auth)

def tweet_collector(username): 
    tweets_df = pd.DataFrame({
        'Timestamp': (),
        'Likes': (),
        'Retweets': (),
        'Text': (),
        'Sentences': (),
        'Sentiment_Total': (),
        'Keywords': ()
    })

    tweets_df = tweets_df[['Timestamp', 'Likes', 'Retweets', 'Text', 'Sentences', 'Sentiment_Total', 'Keywords']]

    recent_tweets = api.user_timeline(screen_name = username, count=100, tweet_mode="extended") # analyzing 100 tweets
    for status in recent_tweets:
        test = status.full_text
        if test[:2] != 'RT': # removing retweets made by the user
            status_data = pd.Series([status.created_at, status.favorite_count, status.retweet_count, status.full_text], 
                                    index=['Timestamp', 'Likes', 'Retweets', 'Text'])
        tweets_df = tweets_df.append(status_data, ignore_index = True)

    tweets_df = tweets_df.drop_duplicates(subset='Text') # just in case, remove any duplicate tweets
    tweets_df = tweets_df.astype('object')
    
    return tweets_df

def keyword_data(tweets_df):
    keywords_dict = {}

    for i in range(len(tweets_df)):
        content = tweets_df.iloc[i]['Text']
        if 'http' in content:
            j = content.index('http')
            content = content[:j] # cleaning text of the tweet by removing the link at the end and newline characters
        content = content.replace('\n', '')
        tweets_df.iloc[i]['Text'] = content

        blob = TextBlob(content)
        tweets_df.iloc[i]['Sentiment_Total'] = blob.sentiment.subjectivity
        sentiments = {}

        for sent in blob.sentences: # generating sentiment polarity values for each sentence in the tweet
            sentiments[str(sent)] = sent.sentiment.subjectivity

        tweets_df.iloc[i]['Sentences'] = sentiments # insert dictionary of sentence: sentiment value into dataframe

        tweets_df.iloc[i]['Timestamp'] = tweets_df.iloc[i]['Timestamp'].to_pydatetime() # convert pandas.tslib.Timestamp object to datetime

        # Keyword extraction goes here
        filtered_words = blob.noun_phrases
    #     print(filtered_words)
        temp = []

        for element in filtered_words:
            for x in range(len(filtered_words)):
    #             print(filtered_words[x])
    #             print(element)
                if element != filtered_words[x] and element in filtered_words[x]:
                    temp.append(element)
                    #filtered_words = [x for x in filtered_words if x != element]
        parts_of_speech = blob.tags
        for element in temp:
            filtered_words = [x for x in filtered_words if x != element]

        for x in range(len(parts_of_speech)):
            if (parts_of_speech[x])[1] == 'NN':
                enter = True
                for element in filtered_words:
                    if (parts_of_speech[x])[0] in element:
                        enter = False
                if enter:
                    if x > 0 and (parts_of_speech[x - 1])[1] == 'PRP$':
                        filtered_words.append((parts_of_speech[x])[0])
        parenthesis = []
        paren_init = 0
        loc_begin = blob.find("(", paren_init)
        loc_end = blob.find(")", paren_init)

        while loc_end >= 0:
            parenthesis.append(blob[loc_begin:loc_end])
            paren_init = loc_end + 1
            loc_begin = blob.find("(", paren_init)
            loc_end = blob.find(")", paren_init)
        #print(parenthesis)

        for element in filtered_words:
            for pelement in parenthesis:
                if element in pelement.lower():
                    filtered_words = [x for x in filtered_words if x != element]
    #     tweets_df.iloc[i]['Keywords'] = filtered_words

        for word in filtered_words:
            separated = TextBlob(word).words
            for j in separated:
                j = j.strip()
                if j.isalpha() and len(j) > 2:
                    if j in keywords_dict:
                        keywords_dict[j][0] += 1
                        keywords_dict[j][1] += tweets_df.iloc[i]['Likes']
                        keywords_dict[j][2] += tweets_df.iloc[i]['Retweets']
                        keywords_dict[j][3] += tweets_df.iloc[i]['Sentiment_Total']
                    else:
                        keywords_dict[j] = [1, tweets_df.iloc[i]['Likes'], tweets_df.iloc[i]['Retweets'], tweets_df.iloc[i]['Sentiment_Total']]

    for key in keywords_dict:
        keywords_dict[key][1] = int(keywords_dict[key][1] / keywords_dict[key][0])
        keywords_dict[key][2] = int(keywords_dict[key][2] / keywords_dict[key][0])
        keywords_dict[key][3] = keywords_dict[key][3] / keywords_dict[key][0]
        
    return keywords_dict

def to_dataframe(keywords_dict):
    keywords_df = pd.DataFrame.from_dict(keywords_dict, orient='index')
    keywords_df.columns = ['Frequency', 'Avg. Likes', 'Avg. Retweets', 'Avg. Sentiment']
    keywords_df.index.name = 'Keywords'
    keywords_df.reset_index(inplace = True)
    keywords_df = keywords_df.sort_values(['Frequency'], ascending = [False], na_position = 'last')
    keywords_df = keywords_df[:30]
    
    return keywords_df

data_df = tweet_collector(username)
keywords = keyword_data(data_df)
key_df = to_dataframe(keywords)

key_df


['democratic party', 'state party', 'young people', 'unity reform', 'country']
['republican', 'tax bill', 'moral abomination', 'taxdebate']
['trans people', 'tdor', 'society']
['disaster relief efforts', 'iran', 'important act']
['iran']
['progressives', 'didn ’ t', 'trump', 'country work']
['trump', 'republican', 'american people', 'democratic party']
['local positions', 'political revolution']
['political revolution', 'political process']
['first-time candidates', 'state legislatures', 'city councils', 'school boards']
['yesterday', 'trump', '’ s', 'american people', 'country']
['’ s election day', 'nyc', 'billdeblasio']
['’ s #', 'electionday', 'jersey', 'democratic ticket']
['’ s time', '@ fortforatlanta', 'electionday find']
['’ s #', 'electionday', 'democratic ticket', 'vote counts']
['’ s election day', 'virginia', 'democratic ticket', 'find']
['sutherland', 'senseless tragedy']
['wall street', 'public colleges', 'universities tuition-free', 'street speculators']
['corporate tax

,Keywords,Frequency,Avg. Likes,Avg. Retweets,Avg. Sentiment
126,health,31,22410,6364,0.383967
127,care,28,24019,6786,0.394366
8,republican,11,17020,4451,0.505177
25,trump,10,28899,8761,0.511472
9,tax,8,9221,3110,0.411979
4,people,8,22943,5090,0.394444
30,political,6,13626,2896,0.320833
166,join,6,2273,566,0.266667
52,virginia,5,7035,1979,0.296667
7,country,5,27450,6395,0.425556
